## RDBMS data modeling

In [38]:
import psycopg2
from sql_queries import user_table_insert, fn_agg, ln_agg
import requests

In [39]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=zylotechdb user=student password=student")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [40]:
# api-endpoint 
URL = "https://reqres.in/api/users?page="
#data requesting from endpoint pages
pages = [1, 2, 3, 4]

In [41]:
for page in pages:
    URL = "https://reqres.in/api/users?page="+str(page)
    print(URL)
    try:
        res = requests.get(url = URL)
        res_json = res.json()
    except requests.exceptions.HTTPError as e:
        print(e)
        
    if res_json['data']:
        users = res_json['data']
        for user in users:
            id1 = user['id']
            email = user['email']
            first_name = user['first_name']
            last_name = user['last_name']
            avatar = user['avatar']
#                 print(id1, email, first_name, last_name, avatar)
            values = (id1,  first_name, last_name,email, avatar, "now()")
    
            try:
                cur.execute(user_table_insert, values)
            except psycopg2.Error as e: 
                print("Error: Inserting Rows")
                print (e) 
    else:
        print("No users list in the API end point.")

https://reqres.in/api/users?page=1
https://reqres.in/api/users?page=2
https://reqres.in/api/users?page=3
https://reqres.in/api/users?page=4


In [42]:
try: 
    cur.execute("SELECT * FROM users;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    #print first_name and last_name
    print(row[2], row[3])
    row = cur.fetchone()

George Bluth
Janet Weaver
Emma Wong
Eve Holt
Charles Morris
Tracey Ramos
Michael Lawson
Lindsay Ferguson
Tobias Funke
Byron Fields
George Edwards
Rachel Howell


In [43]:
# Count of last name starting with same letter:
def aggregation_metrics(query):
    try: 
        cur.execute(query)
        rows = cur.fetchall()
        return rows
    except psycopg2.Error as e: 
        print("Error: ", query)
        print (e)

In [44]:
fn_agg_res = aggregation_metrics(fn_agg)
ln_agg_res = aggregation_metrics(ln_agg)

In [45]:
for row in fn_agg_res:
    print(row)   

('B', 1)
('C', 1)
('E', 2)
('G', 2)
('J', 1)
('L', 1)
('M', 1)
('R', 1)
('T', 2)


In [46]:
for row in ln_agg_res:
    print(row)

('B', 1)
('E', 1)
('F', 3)
('H', 2)
('L', 1)
('M', 1)
('R', 1)
('W', 2)


## NoSQL data modeling

In [47]:
import cassandra
from cassandra.cluster import Cluster
from sql_queries import ac_users_table_create, ac_users_table_insert

In [48]:
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [49]:
try:
    session.set_keyspace('zylotechdb2')
except Exception as e:
    print(e)

In [50]:
try:
    session.execute(ac_users_table_create)
except Exception as e:
    print(e)

In [51]:
def insert_data(session, query):
    '''
        This function is used to insert records into tables
        args:
            session: holds connection
            query: string, query statement to insert into table.
        return: None
    '''
    for page in pages:
        URL = "https://reqres.in/api/users?page="+str(page)
        print(URL)
        try:
            res = requests.get(url = URL)
            res_json = res.json()
        except requests.exceptions.HTTPError as e:
            print(e)

        if res_json['data']:
            users = res_json['data']
            for user in users:
                id1 = user['id']
                email = user['email']
                first_name = user['first_name']
                last_name = user['last_name']
                avatar = user['avatar']
                session.execute(query, (int(id1), str(first_name), str(last_name), str(email), str(avatar)))

In [52]:

# INSERT into the table
insert_data(session, ac_users_table_insert)

https://reqres.in/api/users?page=1
https://reqres.in/api/users?page=2
https://reqres.in/api/users?page=3
https://reqres.in/api/users?page=4


In [66]:

def ac_aggregation_metrics(query):
    
    fc_dic = {}
    try:
        rows = session.execute(query)
        for row in rows:
#             print(row.first_char)
            fn_temp = row.first_char[0].upper()
            if fn_temp in fc_dic:
                fc_dic[fn_temp] +=1
            else:
                fc_dic[fn_temp] = 1
    except Exception as e:
        print(e)
    return fc_dic

In [67]:
#select query
fn_query = """SELECT  first_name as first_char
            FROM ac_users
            """
ln_query = """SELECT  last_name as first_char
            FROM ac_users
            """
fn_result = ac_aggregation_metrics(fn_query)
ln_result = ac_aggregation_metrics(ln_query)

In [68]:
print("fn:", fn_result)
print("ln:", ln_result)

fn: {'C': 1, 'B': 1, 'G': 2, 'L': 1, 'J': 1, 'E': 2, 'M': 1, 'T': 2, 'R': 1}
ln: {'M': 1, 'F': 3, 'E': 1, 'B': 1, 'W': 2, 'H': 2, 'L': 1, 'R': 1}


In [2]:
! pip install schedule

In [ ]:

import functools
import time

import schedule


# This decorator can be applied to
def with_logging(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('LOG: Running job "%s"' % func.__name__)
        result = func(*args, **kwargs)
        print('LOG: Job "%s" completed' % func.__name__)
        return result
    return wrapper

@with_logging
def job():
    print('Hello, World.')

schedule.every(3).seconds.do(job)

while 1:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
session.shutdown()
cluster.shutdown()